In [1]:
import torch
import requests
import numpy as np

from PIL import Image

from transformers import (
    AutoProcessor,
    RTDetrForObjectDetection,
    VitPoseForPoseEstimation,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

url = "http://images.cocodataset.org/val2017/000000000139.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# ------------------------------------------------------------------------
# Stage 1. Detect humans on the image
# ------------------------------------------------------------------------

# You can choose detector by your choice
person_image_processor = AutoProcessor.from_pretrained("PekingU/rtdetr_r50vd_coco_o365")
person_model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd_coco_o365", device_map=device)

inputs = person_image_processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = person_model(**inputs)

results = person_image_processor.post_process_object_detection(
    outputs, target_sizes=torch.tensor([(image.height, image.width)]), threshold=0.3
)
result = results[0]  # take first image results

# Human label refers 0 index in COCO dataset
person_boxes = result["boxes"][result["labels"] == 0]
person_boxes = person_boxes.cpu().numpy()

# Convert boxes from VOC (x1, y1, x2, y2) to COCO (x1, y1, w, h) format
person_boxes[:, 2] = person_boxes[:, 2] - person_boxes[:, 0]
person_boxes[:, 3] = person_boxes[:, 3] - person_boxes[:, 1]

# ------------------------------------------------------------------------
# Stage 2. Detect keypoints for each person found
# ------------------------------------------------------------------------

image_processor = AutoProcessor.from_pretrained("usyd-community/vitpose-plus-small")
model = VitPoseForPoseEstimation.from_pretrained("usyd-community/vitpose-plus-small", device_map=device)

inputs = image_processor(image, boxes=[person_boxes], return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

pose_results = image_processor.post_process_pose_estimation(outputs, boxes=[person_boxes], threshold=0.3)
image_pose_result = pose_results[0]  # results for first image

for i, person_pose in enumerate(image_pose_result):
    print(f"Person #{i}")
    for keypoint, label, score in zip(
        person_pose["keypoints"], person_pose["labels"], person_pose["scores"]
    ):
        keypoint_name = model.config.id2label[label.item()]
        x, y = keypoint
        print(f" - {keypoint_name}: x={x.item():.2f}, y={y.item():.2f}, score={score.item():.2f}")


/Users/nkosinathikhumalo/hackathon/robots/cv-pipeline/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


NameError: name 'inv' is not defined

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define skeleton connections for COCO keypoints
skeleton = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # Head
    (5, 6),  # Shoulders
    (5, 7), (7, 9),  # Left arm
    (6, 8), (8, 10),  # Right arm
    (5, 11), (6, 12),  # Torso
    (11, 12),  # Hips
    (11, 13), (13, 15),  # Left leg
    (12, 14), (14, 16),  # Right leg
]

# Colors for different body parts
colors = plt.cm.rainbow(np.linspace(0, 1, len(skeleton)))

fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(image)

for person_pose in image_pose_result:
    keypoints = person_pose["keypoints"].cpu().numpy()
    scores = person_pose["scores"].cpu().numpy()
    
    # Draw skeleton connections
    for idx, (start, end) in enumerate(skeleton):
        if scores[start] > 0.3 and scores[end] > 0.3:
            x_start, y_start = keypoints[start]
            x_end, y_end = keypoints[end]
            ax.plot([x_start, x_end], [y_start, y_end], 
                   color=colors[idx], linewidth=2, alpha=0.8)
    
    # Draw keypoints
    for i, (kp, score) in enumerate(zip(keypoints, scores)):
        if score > 0.3:
            x, y = kp
            ax.scatter(x, y, c='red', s=40, zorder=5)
            ax.scatter(x, y, c='white', s=15, zorder=6)

ax.set_title("Pose Estimation with ViTPose", fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()